# ENVIRONMENT

In [1]:
import os
import acquire_sso as acquire
import prepare_sso as prepare
import pandas as pd
import numpy as np

# data visualization 
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import statsmodels.api as sm

from datetime import timedelta, datetime
from pylab import rcParams

# to explode the DataFrames and avoid truncation
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

from fbprophet import Prophet

# ACQUIRE

#### _Let's read in the data from the csv file and take a peek at te first five records._

In [2]:
df = acquire.read_data('saws-sso.csv')

In [3]:
df.head()

,SSO_ID,INSPKEY,SERVNO,REPORTDATE,SPILL_ADDRESS,SPILL_ST_NAME,TOTAL_GAL,GALSRET,GAL,SPILL_START,SPILL_STOP,HRS,CAUSE,COMMENTS,ACTIONS,WATERSHED,UNITID,UNITID2,DISCHARGE_TO,DISCHARGE_ROUTE,COUNCIL_DISTRICT,FERGUSON,Month,Year,Week,EARZ_ZONE,Expr1029,PIPEDIAM,PIPELEN,PIPETYPE,INSTYEAR,DWNDPTH,UPSDPTH,Inches_No,RainFall_Less3,SPILL ADDRESS,SewerAssetExp,NUM_SPILLS_COMPKEY,NUM_SPILLS_24MOS,PREVSPILL_24MOS,UNITTYPE,ASSETTYPE,LASTCLND,ResponseTime,ResponseDTTM,Public Notice,TIMEINT,Root_Cause,STEPS_TO_PREVENT,SPILL_START_2,SPILL_STOP_2,HRS_2,GAL_2,SPILL_START_3,SPILL_STOP_3,HRS_3,GAL_3
0,6582,567722.0,NaN,3/10/19,3200,THOUSAND OAKS DR,2100,2100.0,2100.0,3/10/2019 1:16:00 PM,3/10/2019 2:40:00 PM,1.400000,Grease,Spill ContainedReturned to SystemArea Cleaned ...,CLEANED MAIN,SALADO CREEK,66918,66917,STREET,None,NaN,172A2,3,2019,11,0.0,NaN,8.0,16.55,PVC,1997.0,NaN,NaN,NaN,NaN,3200 THOUSAND OAKS DR,NaN,1,1.0,NaN,GRAVITY,Sewer Main,NaN,0.45,10-Mar-19,False,24.0,NaN,NaN,NaN,NaN,0.00,0.0,NaN,NaN,0.0,0.0
1,6583,567723.0,NaN,3/10/19,6804,S FLORES ST,80,0.0,80.0,3/10/2019 2:25:00 PM,3/10/2019 3:45:00 PM,1.333333,Grease,Spill ContainedArea Cleaned and Disinfected,CLEANED MAIN,DOS RIOS,24250,24193,STORMDRAIN,None,3.0,251A3,3,2019,11,0.0,NaN,8.0,157.00,PVC,1988.0,NaN,NaN,NaN,NaN,6804 S FLORES,NaN,1,1.0,NaN,GRAVITY,Sewer Main,NaN,1.08,10-Mar-19,False,120.0,NaN,NaN,NaN,NaN,0.00,0.0,NaN,NaN,0.0,0.0
2,6581,567714.0,NaN,3/9/19,215,AUDREY ALENE DR,79,0.0,10.0,3/9/2019 6:00:00 PM,3/9/2019 7:30:00 PM,1.500000,Structural,Spill ContainedArea Cleaned and DisinfectedFlu...,CLEANED MAIN,DOS RIOS,2822,3351,ALLEY,None,1.0,190E4,3,2019,10,0.0,NaN,8.0,350.00,CP,1955.0,NaN,NaN,NaN,NaN,215 Audrey Alene Dr,NaN,1,1.0,NaN,GRAVITY,Sewer Main,NaN,1.00,09-Mar-19,False,24.0,NaN,NaN,03/10/2019 09:36,03/10/2019 10:45,1.15,69.0,NaN,NaN,0.0,0.0
3,6584,567713.0,NaN,3/9/19,3602,SE MILITARY DR,83,0.0,83.0,3/9/2019 3:37:00 PM,3/9/2019 5:00:00 PM,1.383333,Grease,Spill ContainedArea Cleaned and DisinfectedFlu...,NaN,SALADO CREEK,92804,92805,EASEMENT,None,3.0,252C3,3,2019,10,0.0,NaN,8.0,213.91,PVC,1983.0,NaN,NaN,NaN,NaN,3602 SE MILITARY DR,NaN,1,1.0,NaN,GRAVITY,Sewer Main,NaN,0.55,09-Mar-19,False,120.0,NaN,NaN,NaN,NaN,0.00,0.0,NaN,NaN,0.0,0.0
4,6580,567432.0,NaN,3/6/19,100,PANSY LN,75,0.0,75.0,3/6/2019 9:40:00 AM,3/6/2019 9:55:00 AM,0.250000,Structural,Spill ContainedArea Cleaned and DisinfectedFlu...,CLEANED MAIN,SALADO CREEK,61141,49543,STREET,None,2.0,192A7,3,2019,10,0.0,NaN,12.0,291.90,CP,1952.0,NaN,NaN,NaN,NaN,100 PANSY LN,NaN,2,2.0,15-Dec-18,GRAVITY,Sewer Main,NaN,0.00,06-Mar-19,False,3.0,NaN,NaN,NaN,NaN,0.00,0.0,NaN,NaN,0.0,0.0


# PREPARE

#### _Let's convert the column to lowercase to make them easier to work with and also rename the column names for clarity._

In [4]:
df = prepare.lowercase_and_rename(df)

In [5]:
df.head().T

,0,1,2,3,4
sso_id,6582,6583,6581,6584,6580
inspection_key,567722,567723,567714,567713,567432
service_number,NaN,NaN,NaN,NaN,NaN
report_date,3/10/19,3/10/19,3/9/19,3/9/19,3/6/19
spill_address,3200,6804,215,3602,100
spill_street_name,THOUSAND OAKS DR,S FLORES ST,AUDREY ALENE DR,SE MILITARY DR,PANSY LN
total_gallons,2100,80,79,83,75
gallons_returned,2100,0,0,0,0
gallons_1,2100,80,10,83,75
spill_start_1,3/10/2019 1:16:00 PM,3/10/2019 2:25:00 PM,3/9/2019 6:00:00 PM,3/9/2019 3:37:00 PM,3/6/2019 9:40:00 AM


#### _Let's make copies of the original dataframe before dropping some columns and rows to cover scenarios where we uncover more information about the variables._

In [6]:
df1 = df.copy()
df2 = df.copy()
df3 = df.copy()

#### _Let's prepare df1 for EDA_
0. remove columns that do not add information
0. drop the columns that are no longer needed.
0. Make new variable of whether this incident involved two or more spills within 24 hours
0. Remove redundant columns
0. Rename spill details into simpler names
0. Change column values to lowercase
0. Change address into titlecase
0. Fix the data types
0. Fill nan's
0. Change data type to datetime

In [7]:
df1.shape

(3183, 57)

In [8]:
df1 = prepare.ready_df1(df1)
df1

,report_date,total_gallons,gallons_returned,spill_start,spill_stop,hours,cause,actions,watershed,unit_id_1,unit_id_2,discharge_to,discharge_route,council_district,month,year,week,edwards_zone,pipe_diameter,pipe_length,pipe_type,installation_year,inches_no,rainfall_less_3,num_spills_compkey,num_spills_24mos,unit_type,asset_type,last_cleaned,response_time,response_datetime,public_notice,time_int,root_cause,steps_to_prevent,spill_street_address,multiple_spills
0,2019-03-10,2100,2100.0,3/10/2019 1:16:00 PM,3/10/2019 2:40:00 PM,1.400000,grease,cleaned main,salado creek,66918,66917,street,none,0,3,2019,11,0,8.0,16.550000,pvc,1997,0.00,0.00,1,1,gravity,sewer main,NaT,0.45,2019-03-10,False,24,na,na,3200 Thousand Oaks Dr,False
1,2019-03-10,80,0.0,3/10/2019 2:25:00 PM,3/10/2019 3:45:00 PM,1.333333,grease,cleaned main,dos rios,24250,24193,stormdrain,none,3,3,2019,11,0,8.0,157.000000,pvc,1988,0.00,0.00,1,1,gravity,sewer main,NaT,1.08,2019-03-10,False,120,na,na,6804 S Flores St,False
2,2019-03-09,79,0.0,3/9/2019 6:00:00 PM,3/9/2019 7:30:00 PM,1.500000,structural,cleaned main,dos rios,2822,3351,alley,none,1,3,2019,10,0,8.0,350.000000,cp,1955,0.00,0.00,1,1,gravity,sewer main,NaT,1.00,2019-03-09,False,24,na,na,215 Audrey Alene Dr,True
3,2019-03-09,83,0.0,3/9/2019 3:37:00 PM,3/9/2019 5:00:00 PM,1.383333,grease,na,salado creek,92804,92805,easement,none,3,3,2019,10,0,8.0,213.910000,pvc,1983,0.00,0.00,1,1,gravity,sewer main,NaT,0.55,2019-03-09,False,120,na,na,3602 Se Military Dr,False
4,2019-03-06,75,0.0,3/6/2019 9:40:00 AM,3/6/2019 9:55:00 AM,0.250000,structural,cleaned main,salado creek,61141,49543,street,none,2,3,2019,10,0,12.0,291.900000,cp,1952,0.00,0.00,2,2,gravity,sewer main,NaT,0.00,2019-03-06,False,3,na,na,100 Pansy Ln,False
5,2019-03-05,250,0.0,3/5/2019 2:22:00 PM,3/5/2019 2:32:00 PM,0.166667,grease,cleaned main,dos rios,38907,26117,street,none,3,3,2019,10,0,8.0,315.000000,rl,1992,0.00,0.00,2,2,gravity,sewer main,NaT,0.00,2019-03-05,False,12,na,na,3200 S Hackberry St,False
6,2019-03-02,73,0.0,3/2/2019 1:42:00 PM,3/2/2019 2:55:00 PM,1.216667,grease,cleaned main,medio creek,85120,85363,drainage culvert,none,4,3,2019,9,0,8.0,264.470000,pvc,1985,0.00,0.00,1,1,gravity,sewer main,NaT,0.73,2019-03-02,False,120,grease,"Increase FCS,",9910 Sugarloaf Dr,False
7,2019-03-01,76,0.0,3/1/2019 6:34:00 PM,3/1/2019 7:50:00 PM,1.266667,grease,cleaned main,dos rios,26128,24334,stormdrain,none,3,3,2019,9,0,8.0,60.000000,rl,2015,0.00,0.00,1,1,gravity,sewer main,NaT,0.43,2019-03-01,False,120,na,"Increase FCS,",3507 Piedmont Ave,False
8,2019-02-26,3750,0.0,2/26/2019 9:00:00 AM,2/26/2019 10:15:00 AM,1.250000,structural,cleaned main,leon creek,47292,47293,stormdrain,none,7,2,2019,9,0,8.0,175.390000,cp,1956,0.00,0.00,1,1,gravity,sewer main,NaT,0.00,2019-02-26,False,120,structural,"Design Request,",349 Alicia,False
9,2019-02-26,66,0.0,2/26/2019 5:24:00 PM,2/26/2019 6:30:00 PM,1.100000,grease,cleaned main,dos rios,14241,14896,street,none,1,2,2019,9,0,8.0,194.100000,pvc,1992,0.00,0.00,1,1,gravity,sewer main,NaT,0.43,2019-02-26,False,120,debris,"Increase FCS,",1502 W Mistletoe Ave,False


In [9]:
df1.dtypes

report_date             datetime64[ns]
total_gallons                    int64
gallons_returned               float64
spill_start                     object
spill_stop                      object
hours                          float64
cause                           object
actions                         object
watershed                       object
unit_id_1                       object
unit_id_2                       object
discharge_to                    object
discharge_route                 object
council_district                 int64
month                            int64
year                             int64
week                             int64
edwards_zone                     int64
pipe_diameter                  float64
pipe_length                    float64
pipe_type                       object
installation_year                int64
inches_no                      float64
rainfall_less_3                float64
num_spills_compkey               int64
num_spills_24mos         

#### _Let's assign a variable with all numerical column names._

In [10]:
df1_numerical_columns = list(df1.select_dtypes(include=[np.number]).columns.values)
df1_numerical_columns

['total_gallons',
 'gallons_returned',
 'hours',
 'council_district',
 'month',
 'year',
 'week',
 'edwards_zone',
 'pipe_diameter',
 'pipe_length',
 'installation_year',
 'inches_no',
 'rainfall_less_3',
 'num_spills_compkey',
 'num_spills_24mos',
 'time_int']

#### _Let's assign a variable with all non-numerical column names._

In [11]:
df1_non_numerical_columns = list(df1.select_dtypes(exclude=[np.number]).columns.values)
df1_non_numerical_columns

['report_date',
 'spill_start',
 'spill_stop',
 'cause',
 'actions',
 'watershed',
 'unit_id_1',
 'unit_id_2',
 'discharge_to',
 'discharge_route',
 'pipe_type',
 'unit_type',
 'asset_type',
 'last_cleaned',
 'response_time',
 'response_datetime',
 'public_notice',
 'root_cause',
 'steps_to_prevent',
 'spill_street_address',
 'multiple_spills']

### Let's get an idea of how often these pipes are cleaned

In [12]:
df1.set_index('last_cleaned').groupby(pd.Grouper(freq='M')).count()

,report_date,total_gallons,gallons_returned,spill_start,spill_stop,hours,cause,actions,watershed,unit_id_1,unit_id_2,discharge_to,discharge_route,council_district,month,year,week,edwards_zone,pipe_diameter,pipe_length,pipe_type,installation_year,inches_no,rainfall_less_3,num_spills_compkey,num_spills_24mos,unit_type,asset_type,response_time,response_datetime,public_notice,time_int,root_cause,steps_to_prevent,spill_street_address,multiple_spills
last_cleaned,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2009-03-31,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,0,9,9,9,9,9,9
2009-04-30,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,0,28,28,28,28,28,28
2009-05-31,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,0,26,26,26,26,26,26
2009-06-30,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,0,25,25,25,25,25,25
2009-07-31,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,0,14,14,14,14,14,14
2009-08-31,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,0,20,20,20,20,20,20
2009-09-30,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,0,26,26,26,26,26,26
2009-10-31,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,0,41,41,41,41,41,41
2009-11-30,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,2,29,29,29,29,29,29


In [13]:
df1.last_cleaned.value_counts(dropna=False)

NaT           2082
2009-09-23       6
2009-10-22       5
2009-08-22       5
2012-05-11       5
2009-11-25       4
2009-12-07       4
2009-04-18       4
2012-02-29       4
2010-01-07       4
2010-06-01       4
2009-10-20       4
2009-11-23       4
2013-06-19       4
2009-12-28       4
2009-12-22       3
2010-02-11       3
2013-11-22       3
2012-11-19       3
2010-08-09       3
2010-10-04       3
2011-03-17       3
2014-06-18       3
2013-10-17       3
2010-02-22       3
2009-06-24       3
2010-02-15       3
2011-12-07       3
2010-01-04       3
2010-10-07       3
2009-03-17       3
2014-04-11       3
2011-02-12       3
2010-08-27       3
2011-03-21       3
2010-02-05       3
2011-11-10       3
2011-10-19       3
2009-12-10       3
2009-05-01       3
2011-05-26       3
2010-08-26       3
2009-11-12       3
2010-04-15       3
2009-08-29       3
2010-09-21       3
2010-03-15       3
2010-11-18       3
2011-08-03       3
2010-03-08       3
2010-02-19       3
2009-12-17       3
2013-08-22  

In [15]:
prepare.missing_values_col(df1)

,num_missing,missing_percentage,num_empty,empty_percentage,nan_count,nan_percentage
report_date,0,0.000000,0,0.000000,0,0.0
total_gallons,0,0.000000,0,0.000000,0,0.0
gallons_returned,0,0.000000,0,0.000000,0,0.0
spill_start,0,0.000000,0,0.000000,0,0.0
spill_stop,0,0.000000,0,0.000000,0,0.0
hours,0,0.000000,0,0.000000,0,0.0
cause,0,0.000000,0,0.000000,0,0.0
actions,0,0.000000,0,0.000000,0,0.0
watershed,0,0.000000,0,0.000000,0,0.0
unit_id_1,0,0.000000,0,0.000000,0,0.0


### Let's feature engineer a column to show how many months have passed since the last cleaning and the current spill.

In [17]:
df2 = prepare.ready_df1(df2)
df2_numerical_columns = list(df2.select_dtypes(include=[np.number]).columns.values)
df2_non_numerical_columns = list(df2.select_dtypes(exclude=[np.number]).columns.values)
prepare.missing_values_col(df2)

,num_missing,missing_percentage,num_empty,empty_percentage,nan_count,nan_percentage
report_date,0,0.000000,0,0.000000,0,0.0
total_gallons,0,0.000000,0,0.000000,0,0.0
gallons_returned,0,0.000000,0,0.000000,0,0.0
spill_start,0,0.000000,0,0.000000,0,0.0
spill_stop,0,0.000000,0,0.000000,0,0.0
hours,0,0.000000,0,0.000000,0,0.0
cause,0,0.000000,0,0.000000,0,0.0
actions,0,0.000000,0,0.000000,0,0.0
watershed,0,0.000000,0,0.000000,0,0.0
unit_id_1,0,0.000000,0,0.000000,0,0.0


In [18]:
df2['months_elapsed'] = ((df2.report_date - df2.last_cleaned)/np.timedelta64(1, 'M'))

### Drop NaN's in the two columns we want to work with right now

In [21]:
df2.dropna(subset=['last_cleaned', 'months_elapsed'], inplace=True)

In [22]:
df2.shape

(1101, 38)